In [ ]:
!pip install shapely

In [ ]:
!pip install schedule

OVAL GEOJSON GENERATOR


In [ ]:
import json
import requests
import schedule
import time
import os
from datetime import datetime, timedelta
from shapely.geometry import Polygon
from shapely.geometry.polygon import orient

# Function to convert RGB to Hex
def rgb_to_hex(rgb_str):
    """
    Convert RGB color string in format 'rgb(r, g, b)' to HEX color code.

    Parameters:
    rgb_str (str): RGB color string in format 'rgb(r, g, b)'

    Returns:
    str: HEX color code
    """
    # Extract numeric values from the string
    rgb_values = rgb_str.split('(')[1].split(')')[0].split(',')
    r = int(rgb_values[0].strip())
    g = int(rgb_values[1].strip())
    b = int(rgb_values[2].strip())


    # Clamp RGB values within 0-255 range
    r = max(0, min(r, 255))
    g = max(0, min(g, 255))
    b = max(0, min(b, 255))

    # Convert RGB to HEX
    return '#{:02x}{:02x}{:02x}'.format(r, g, b)



def douglas_peucker(coords, tolerance):
    """Douglas-Peucker algorithm for simplifying coordinates."""
    from math import sqrt

    def perpendicular_distance(pt, start, end):
        # Calculate the perpendicular distance from pt to line defined by start-end
        if start == end:
            return sqrt((pt[0] - start[0]) ** 2 + (pt[1] - start[1]) ** 2)
        else:
            numerator = abs((end[1] - start[1]) * pt[0] - (end[0] - start[0]) * pt[1] + end[0] * start[1] - end[1] * start[0])
            denominator = sqrt((end[1] - start[1]) ** 2 + (end[0] - start[0]) ** 2)
            return numerator / denominator

    def recursive_douglas_peucker(coords, tolerance, start=0, end=None):
        if end is None:
            end = len(coords) - 1

        if end <= start + 1:
            return [coords[start], coords[end]]

        max_distance = 0
        max_index = 0

        for i in range(start + 1, end):
            distance = perpendicular_distance(coords[i], coords[start], coords[end])
            if distance > max_distance:
                max_distance = distance
                max_index = i

        if max_distance >= tolerance:
            first_part = recursive_douglas_peucker(coords, tolerance, start, max_index)
            second_part = recursive_douglas_peucker(coords, tolerance, max_index, end)
            return first_part[:-1] + second_part
        else:
            return [coords[start], coords[end]]

    return recursive_douglas_peucker(coords, tolerance)


# Function to ensure proper order, closure, and orientation of coordinates
def process_coordinates(coords):
    # Ensure proper order and closure
    if coords[0] != coords[-1]:
        coords.append(coords[0])  # Close the polygon if not closed already

    # Orient the polygon
    poly = Polygon(coords)
    oriented_coords = list(orient(poly).exterior.coords)

    oriented_coords.reverse()

    # Convert coordinates to the correct format for GeoJSON
    return [[round(x, 6), round(y, 6)] for x, y in oriented_coords]

# Function to check if the polygon needs to be reversed (for poles)
def is_polygon_reversed(coords):
    # Determine if the polygon should be reversed based on its centroid
    # Example logic: Check if the centroid latitude is above or below a certain threshold
    centroid_y = sum(coord[1] for coord in coords) / len(coords)
    if centroid_y > 0:  # Adjust this threshold as needed
        return True
    else:
        return False


# Function to simplify coordinates using Douglas-Peucker algorithm
def simplify_coordinates(coords, tolerance=0.001):
    simplified_coords = douglas_peucker(coords, tolerance)
    return simplified_coords

# Input and output file paths
input_file = 'C:/Users/alundkvi/Documents/work/scripts/oval-dataTimeTest.json'
output_file = 'C:/Users/alundkvi/Documents/work/scripts/oval_asset_output_not_simplified/output.geojson'

# Function to process each entry and generate GeoJSON-like asset
def process_entry(entry):
    fill_color = entry.get('fill_color', None)
    paths = entry.get('paths', [])

    # Validate input structure
    if not fill_color or not paths:
        return None

    # Convert RGB to Hex
    fill_color_hex = rgb_to_hex(fill_color)

    optimized_paths = []

    # Process each path
    for path in paths:
        optimized_path = []
        
        # Modify each coordinate in the path and optimize
        for coord in path:
            lat = coord.get('lat', None)
            lng = coord.get('lng', None)

            # Skip if coordinates are in the northern hemisphere
            if lat is not None and lat > 0:
                continue

            # Swap lat and lng, subtract 400 from lng
            if lat is not None and lng is not None:
                optimized_path.append([
                    lng - 360,
                    lat
                ])

        # Simplify coordinates if there are valid points in optimized_path
        if optimized_path:
            simplified_coords = simplify_coordinates(optimized_path)
            final_coords = process_coordinates(simplified_coords)
            optimized_paths.append(final_coords)

    # Construct GeoJSON-like asset structure if optimized_paths is not empty
    if optimized_paths:
        asset = {
            "type": "Feature",
            "properties": {
                "stroke": fill_color_hex,
                "stroke-width": 2,
                "stroke-opacity": 1,
                "fill": fill_color_hex,
                "fill-opacity": 0.75
            },
            "geometry": {
                "type": "Polygon",
                "coordinates": optimized_paths
            }
        }

        return asset
    else:
        return None

# Function to fetch data from URL
def fetch_data_and_process(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data from {url}")
        return []   

# Function to run the process for each 15-minute interval
#def run_process():
# Define the time range
start_time = datetime(2024, 5, 10, 15, 0)
end_time = datetime(2024, 5, 12, 0, 0)

current_time = start_time
interval_minutes = 15

while current_time <= end_time:
    formatted_time = current_time.strftime("%Y-%m-%d-%H-%M")
    folder_name = formatted_time.replace(':', '-')
    output_folder = f'C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/{folder_name}/'

    # Create output folder if it doesn't exist
    try:
        os.makedirs(output_folder, exist_ok=True)
    except OSError as e:
        print(f"Error creating directory {output_folder}: {e}")
        continue
    # Fetch data from URL
    url = f'https://aurorasaurus.org/oval-data?end_date={current_time.strftime("%Y-%m-%dT%H:%M%z")}&local_offset=0'
    data = fetch_data_and_process(url)
    
    display(f"URL: {url}")

    if data:
        # Process each entry and generate GeoJSON-like asset
        for index, entry in enumerate(data):
            asset = process_entry(entry)
            if asset:
                # Generate output file path
                output_file = f'{output_folder}output{index + 1}.geojson'

                # Write GeoJSON-like asset to output file
                try:
                    with open(output_file, 'w') as out_f:
                        json.dump(asset, out_f, indent=2)
                    print(f'Processed entry {index + 1} saved to {output_file}')
                except Exception as e:
                    print(f"Error writing to {output_file}: {e}")
            else:
                print(f'Skipped entry {index + 1} due to missing or invalid data')

    # Move to the next interval
    current_time += timedelta(minutes=interval_minutes)

# Schedule the job every 15 minutes
#schedule.every(1).minutes.do(run_process)

# Run indefinitely
#while True:
#    schedule.run_pending()
#    time.sleep(1)

OVAL ASSET GENERATOR

In [ ]:
import os
import json
from datetime import datetime

# Directory where GeoJSON files are stored
output_directory = 'C:/Users/alundkvi/Documents/work/scripts/oval_output_only_south/'

# Output directory for .asset files
asset_output_directory = 'C:/Users/alundkvi/Documents/work/scripts/oval_only_south_assets/'

# Function to generate .asset content for a folder of GeoJSON files
def generate_asset(folder_path):
    # Extract date and time from folder name
    folder_name = os.path.basename(folder_path)
    date_time_str = folder_name.replace('_', ' ')
    date_time = datetime.strptime(date_time_str, "%Y-%m-%d %H-%M-%S")

    # Define Lua script content template
    lua_script = f"""
local earth = asset.require("scene/solarsystem/planets/earth/earth")

local geojsonPath = asset.resource("C:/Users/alundkvi/Documents/work/OpenSpace/user/data/assets/aurorasaurus/geojson/oval_only_south_geojsons/{folder_name}/")

"""
# Calculate start and end times for the 15-minute interval
    start_time_str = date_time.strftime("%Y %B %d %H:%M:%S")
    end_time = date_time + timedelta(minutes=15)
    end_time_str = end_time.strftime("%Y %B %d %H:%M:%S")
    # Process each GeoJSON file in the folder
    file_index = 1
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith('.geojson'):
            file_path = os.path.join(folder_path, filename)
            # Generate Lua script for each GeoJSON file
            lua_script += f"""
local AuroraOval{file_index} = {{
  Identifier = "AuroraOval_{date_time_str}_{file_index}",
  File = geojsonPath .. "{filename}",
  TimeFrame = {{
    Type = "TimeFrameInterval",
    Start = "{start_time_str}",
    End = "{end_time_str}"
  }},
  HeightOffset = 75000,
  Name = "AuroraOval_{date_time_str}_{file_index}"
}}

asset.onInitialize(function()

  openspace.globebrowsing.addGeoJson(earth.Earth.Identifier, AuroraOval{file_index})

end)

asset.onDeinitialize(function()

  openspace.globebrowsing.deleteGeoJson(earth.Earth.Identifier, AuroraOval{file_index})

end)

"""
            file_index += 1

            # Move to the next 15-minute interval
            date_time = end_time

    # Write Lua script to .asset file in asset_output_directory
    asset_filename = f'{folder_name}.asset'
    asset_filepath = os.path.join(asset_output_directory, asset_filename)
    with open(asset_filepath, 'w') as asset_file:
        asset_file.write(lua_script)

    print(f'Generated {asset_filename} in {asset_output_directory}')

# Ensure asset output directory exists
os.makedirs(asset_output_directory, exist_ok=True)

# Iterate through each folder in output_directory
for folder_name in sorted(os.listdir(output_directory)):
    folder_path = os.path.join(output_directory, folder_name)
    if os.path.isdir(folder_path):
        generate_asset(folder_path)

In [ ]:
from datetime import datetime, timedelta

# Define start and end datetime
start_datetime = datetime(2024, 5, 10, 15, 0, 0)
end_datetime = datetime(2024, 5, 12, 12, 0, 0)

# Define interval duration (15 minutes)
interval = timedelta(minutes=15)

# Function to generate required asset paths
def generate_asset_requirements(start, end, interval):
    current = start
    while current <= end:
        asset_path = f'asset.require("./oval_only_south_assets/{current.strftime("%Y-%m-%d_%H-%M-%S")}")'
        print(asset_path)
        current += interval

# Generate asset requirements
generate_asset_requirements(start_datetime, end_datetime, interval)


VIEWLINE GEOJSON GENERATOR

In [ ]:
import requests
import json
from datetime import datetime, timedelta
import os

# Function to fetch data from URL and adjust coordinates
def fetch_northern_coordinates(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        northern_coordinates = data.get('northern', [])
        # Adjust lng coordinates by removing 360
        adjusted_coordinates = [(item['lng'] - 360, item['lat']) for item in northern_coordinates]
        return adjusted_coordinates
    else:
        print(f"Failed to fetch data from {url}")
        return []
    
    
# Function to fetch data from URL and adjust coordinates
def fetch_southern_coordinates(url):
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        southern_coordinates = data.get('southern', [])
        # Adjust lng coordinates by removing 360
        adjusted_coordinates = [(item['lng'] - 360, item['lat']) for item in southern_coordinates]
        return adjusted_coordinates
    else:
        print(f"Failed to fetch data from {url}")
        return []

# Function to create LineString GeoJSON and save to file
def create_geojson_file(coordinates, output_path):
    if not os.path.exists(os.path.dirname(output_path)):
        os.makedirs(os.path.dirname(output_path))
        
    feature_collection = {
        "type": "FeatureCollection",
        "features": []
    }

    line_feature = {
        "type": "Feature",
        "properties": {
            "name": "Aurora viewline",
            "stroke": "#FF5733"
        },
        "geometry": {
            "type": "LineString",
            "coordinates": coordinates
        }
    }

    feature_collection["features"].append(line_feature)

    with open(output_path, 'w') as f:
        json.dump(feature_collection, f, indent=4)

# Define output directory
output_directory = "C:/Users/alundkvi/Documents/work/scripts/southern_viewline_asset_output/"

# Define start and end datetime
start_datetime = datetime(2024, 5, 10, 15, 0, 0)
end_datetime = datetime(2024, 5, 12, 0, 0, 0)
interval = timedelta(minutes=15)

# Iterate through each 15-minute interval
current_datetime = start_datetime
while current_datetime <= end_datetime:
    # Construct URL for the current interval
    formatted_datetime = current_datetime.strftime("%Y-%m-%dT%H:%M:%S%z")
    url = f"https://aurorasaurus.org/view-lines?end_date={formatted_datetime}&format=json&local_offset=0"

    # Fetch northern coordinates from URL
    #northern_coordinates = fetch_northern_coordinates(url)

    # Fetch southern coordinates from URL
    southern_coordinates = fetch_southern_coordinates(url)

    # Create output filename based on datetime
    output_filename = current_datetime.strftime("%Y-%m-%d_%H-%M-%S") + ".geojson"
    output_path = os.path.join(output_directory, output_filename)

    # Create GeoJSON file with northern coordinates in the specified format
    #create_geojson_file(northern_coordinates, output_path)
    
    # Create GeoJSON file with southern coordinates in the specified format
    create_geojson_file(southern_coordinates, output_path)

    # Move to the next interval
    current_datetime += interval


VIEWLINE ASSET GENERATOR


In [ ]:
import os
import json
from datetime import datetime

# Directory where GeoJSON files are stored
output_directory = 'C:/Users/alundkvi/Documents/work/scripts/southern_viewline_asset_output/'

# Output directory for .asset files
asset_output_directory = 'C:/Users/alundkvi/Documents/work/scripts/southern_viewline_asset/'

# Function to generate .asset content for a folder of GeoJSON files
def generate_asset(folder_path):
    # Define Lua script content template
    lua_script = f"""
local earth = asset.require("scene/solarsystem/planets/earth/earth")

local geojsonPath = asset.resource("geojson/southern_viewline_geojsons/")

"""

    # Process each GeoJSON file in the folder
    file_index = 1
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith('.geojson'):
             # Extract date and time from file name
            date_time_str = filename.replace('_', ' ')
            date_time_str = date_time_str.replace('.geojson', '')
            date_time = datetime.strptime(date_time_str, "%Y-%m-%d %H-%M-%S")

        # Calculate start and end times for the 15-minute interval
            start_time_str = date_time.strftime("%Y %B %d %H:%M:%S")
            end_time = date_time + timedelta(minutes=15)
            end_time_str = end_time.strftime("%Y %B %d %H:%M:%S")
            file_path = os.path.join(folder_path, filename)
            # Generate Lua script for each GeoJSON file
            lua_script += f"""
local southern_viewline_{file_index} = {{
  Identifier = "southern_viewline_{date_time_str}_{file_index}",
  File = geojsonPath .. "{filename}",
  TimeFrame = {{
    Type = "TimeFrameInterval",
    Start = "{start_time_str}",
    End = "{end_time_str}"
  }},
  HeightOffset = 75000,
  Name = "southern_viewline_{date_time_str}_{file_index}"
}}

"""
            file_index += 1

            # Move to the next 15-minute interval
            date_time = end_time


    lua_script += f"""
asset.onInitialize(function()
"""

    for i in range(1, file_index):
      lua_script += f"""openspace.globebrowsing.addGeoJson(earth.Earth.Identifier, southern_viewline_{i})
"""
    lua_script += f"""
end)
"""


    lua_script += f"""
asset.onDeinitialize(function()
"""
    for i in range(1, file_index):
      lua_script += f"""openspace.globebrowsing.deleteGeoJson(earth.Earth.Identifier, southern_viewline_{i})
"""
    lua_script += f"""
end)

"""
    # Write Lua script to .asset file in asset_output_directory
    asset_filename = 'southernViewline.asset'
    asset_filepath = os.path.join(asset_output_directory, asset_filename)
    with open(asset_filepath, 'w') as asset_file:
        asset_file.write(lua_script)

    print(f'Generated {asset_filename} in {asset_output_directory}')

# Ensure asset output directory exists
os.makedirs(asset_output_directory, exist_ok=True)

generate_asset(output_directory)

In [ ]:
!pip install geopandas

In [ ]:
!pip install matplotlib geopandas

In [ ]:
!pip install GDAL


In [ ]:
!pip install Fiona


In [ ]:
!pip install Shapely

In [ ]:
!pip uninstall numpy

In [ ]:
!pip install Fiona

In [ ]:
!pip install numpy==1.21.2

LAYER OVAL THIS IS THE ACTUAL ONE

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import geopandas as gpd
import numpy as np
import os
from PIL import Image

# Root directory containing folders with GeoJSON files
root_dir = 'C:/Users/alundkvi/Documents/work/scripts/oval_geojsons_south/temp/'

# Function to determine the shift needed for longitudes to fit within -180 to 180
def calculate_shift_amounts(coords):
    longitudes = coords[:, 0]
    shift = 0
    plus = True
    abs_max = abs(np.max(longitudes))
    abs_min = abs(np.min(longitudes))

    if abs_max > abs_min and np.max(longitudes) > 180:
        shift = np.max(longitudes) - 180
        plus = False
    elif abs_max < abs_min and np.min(longitudes) < -180:
        shift = -180 - np.min(coords[:, 0])
        plus = True

    return shift, plus

def shift_image(image_path, degrees, output_path):
    # Load the image
    image = Image.open(image_path).convert("RGBA")  # Ensure the image is in RGBA mode to handle transparency
    width, height = image.size
    
    # Calculate how many pixels to shift
    pixels_per_degree = width / 360
    shift_pixels = int(degrees * pixels_per_degree)

    # Create a new image with transparent background
    new_image = Image.new('RGBA', (width, height), (0, 0, 0, 0))

    # Split the image into two parts, considering transparency
    right_part = image.crop((width - shift_pixels, 0, width, height))
    left_part = image.crop((0, 0, width - shift_pixels, height))

    # Paste the shifted parts onto the new image
    new_image.paste(right_part, (0, 0), right_part)  # Use the image itself as a mask to preserve transparency
    new_image.paste(left_part, (shift_pixels, 0), left_part)  # Use the image itself as a mask to preserve transparency

    # Save the result
    new_image.save(output_path)
    
def shift_image_left(image_path, degrees, output_path):
    # Load the image
    image = Image.open(image_path).convert("RGBA")  # Ensure the image is in RGBA mode to handle transparency
    width, height = image.size
    
    # Calculate how many pixels to shift
    pixels_per_degree = width / 360
    shift_pixels = int(degrees * pixels_per_degree)

    # Create a new image with transparent background
    new_image = Image.new('RGBA', (width, height), (0, 0, 0, 0))

    # Split the image into two parts, considering transparency
    left_part = image.crop((0, 0, shift_pixels, height))
    right_part = image.crop((shift_pixels, 0, width, height))

    # Paste the shifted parts onto the new image
    new_image.paste(left_part, (width - shift_pixels, 0), left_part)  # Paste left part at the end to wrap around
    new_image.paste(right_part, (0, 0), right_part)  # Paste right part in its original position

    # Save the result
    new_image.save(output_path)

# Function to adjust coordinates based on shift amounts
def adjust_longitudes(coords, shift, plus):
    if plus == False:
        coords[:, 0] -= shift
    else:
        coords[:, 0] += shift
    return coords

# Iterate over all subdirectories and their files
for dirpath, _, filenames in os.walk(root_dir):
    geojson_files = [f for f in filenames if f.endswith('.geojson')]

    if not geojson_files:
        continue

    # Process each GeoJSON file to determine the maximum and minimum longitudes
    all_coords = []
    for filename in geojson_files:
        geojson_file = os.path.join(dirpath, filename)
        gdf = gpd.read_file(geojson_file)
        polygon_coords = np.array(gdf['geometry'][0].exterior.coords.xy).T
        all_coords.append(polygon_coords)

    # Flatten list of coordinates
    all_coords = np.vstack(all_coords)

    # Calculate the shift amounts
    shift, plus = calculate_shift_amounts(all_coords)

    print(dirpath)
    print(shift)
    #print(" ")

    # Apply the shift to each file
    for filename in geojson_files:
        geojson_file = os.path.join(dirpath, filename)
        gdf = gpd.read_file(geojson_file)
        polygon_coords = np.array(gdf['geometry'][0].exterior.coords.xy).T

        # Adjust longitudes
        adjusted_coords = adjust_longitudes(polygon_coords, shift, plus)

        # Extract fill color from GeoJSON properties
        fill_color = gdf['fill'][0]

        # Create a figure and axis with specific dimensions
        fig, ax = plt.subplots(figsize=(72, 36))

        # Create a polygon patch for the adjusted coordinates
        if len(adjusted_coords) > 0:
            polygon_patch = Polygon(adjusted_coords, closed=True, edgecolor=fill_color, facecolor=fill_color, alpha=1)
            ax.add_patch(polygon_patch)

        # Set axis limits and aspect ratio
        ax.set_xlim(-180, 180)
        ax.set_ylim(-90, 90)
        ax.set_aspect('equal')

        # Remove axes
        ax.axis('off')

        # Output image file path
        output_file = os.path.splitext(filename)[0] + '.png'
        output_path = os.path.join(dirpath, output_file)

        # Save the image
        plt.savefig(output_path, bbox_inches='tight', pad_inches=0, transparent=True)
        plt.close()  # Close the current figure to free up memory

        shift_image_left(output_path, shift, output_path)
